# Project Title: Property Usage
## Data cleansing for zoning geoinfo
## By Go Code CO team "Tech Slope" - Aaron, Adam, Dex, Juli, Leo, Marc
### Version 0.7
### Last update: May 11, 2019

In [0]:
### Initiate
!pip install geopandas

import pandas as pd
import geopandas as gpd
import time
import numpy as np
import matplotlib as plt
from matplotlib.path import Path
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

## Task 1 Load and Clean Zones

In [0]:
### Load Opportunity Zones
layer_zoning_opp='OppZonesCO.json'
df_opp = gpd.read_file(layer_zoning_opp)
df_opp.head() #126 rows. GEOID is the key

,FID,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,GEOID_1,Nominated,Certified,countyFips,OZcnty,geometry
0,0,08,041,002300,08041002300,23,Census Tract 23,G5020,S,3164176,0,+38.8272733,-104.8263887,08041002300,Yes,Yes,41,EL PASO,"POLYGON ((-104.836294 38.822963, -104.8363 38...."
1,23,08,041,004008,08041004008,40.08,Census Tract 40.08,G5020,S,40118194,21258,+38.7959007,-104.7044599,08041004008,Yes,Yes,41,EL PASO,"POLYGON ((-104.730221 38.794775, -104.730207 3..."
2,29,08,037,000503,08037000503,5.03,Census Tract 5.03,G5020,S,67706269,243235,+39.6496146,-106.5019517,08037000503,Yes,Yes,37,EAGLE,"POLYGON ((-106.55577 39.641816, -106.55575 39...."
3,44,08,041,005000,08041005000,50,Census Tract 50,G5020,S,4962692,8832,+38.8509467,-104.7109360,08041005000,Yes,Yes,41,EL PASO,"POLYGON ((-104.72067 38.842179, -104.720672 38..."
4,60,08,045,951702,08045951702,9517.02,Census Tract 9517.02,G5020,S,11361943,0,+39.5347779,-107.3102238,08045951702,Yes,Yes,45,GARFIELD,"POLYGON ((-107.329562 39.546273, -107.32971 39..."


In [0]:
### Load Enterprise Zone
layer_zoning_ent='EnterpriseZones_2019.json'
df_ent = gpd.read_file(layer_zoning_ent)
df_ent.head() #5 rows. No keys

,Name,Description,geometry
0,,,"(POLYGON ((-104.789909 39.986807, -104.790806 ..."
1,,,"(POLYGON ((-105.551328 39.567303, -105.551215 ..."
2,,,"(POLYGON ((-105.006595 39.696724, -105.006509 ..."
3,,,"(POLYGON ((-102.049806 39.574058, -102.049764 ..."
4,,,"(POLYGON ((-105.108609 39.914137, -105.108439 ..."


In [0]:
### Load Zoning
layer_zoning_gj='Zoning_GJ.json'
df_zoning_gj = gpd.read_file(layer_zoning_gj)
df_zoning_gj.head()

,Name,Description,geometry
0,C-2,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","(POLYGON ((-108.451355 39.099361, -108.451735 ..."
1,RSF-4,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","(POLYGON ((-108.513754 39.030293, -108.514123 ..."
2,PUD,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","(POLYGON ((-108.512976 39.030322, -108.512976 ..."
3,RSF-4,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","(POLYGON ((-108.512038 39.028617, -108.512492 ..."
4,PUD,"<html xmlns:fo=""http://www.w3.org/1999/XSL/For...","(POLYGON ((-108.652886 39.122644, -108.654471 ..."


## Task 2 Find zoning for each Mesa County address


In [0]:
## Match Opportunity Zones
df_mesazone = pd.read_csv('Address_Mesa_Cleaned.csv',dtype={'StreetNumber':str}) ## 49849 rows
df_mesazone['OppZoneGeoID']='0'
df_mesazone['EntZone']=False
df_mesazone['ZoningCode']='0'
df_mesazone.head()

,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
0,38.523159,-108.889763,56500,HWY 141,NaN,GATEWAY,CO,81522,56500 HWY 141,0,False,0
1,38.534833,-108.896495,55002,HWY 141,NaN,GATEWAY,CO,81522,55002 HWY 141,0,False,0
2,38.535193,-108.838723,34165,S 12 8/10 RD,NaN,WHITEWATER,CO,81527,34165 S 12 8/10 RD,0,False,0
3,38.541657,-108.567979,30200,UNCOMPAHGRE DIVIDE RD,NaN,WHITEWATER,CO,815XX,30200 UNCOMPAHGRE DIVIDE RD,0,False,0
4,38.563368,-108.921568,52248,HWY 141,NaN,GATEWAY,CO,81522,52248 HWY 141,0,False,0


In [0]:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_opp)): 
        if df_opp['geometry'][j].contains(point):
            df_mesazone['OppZoneGeoID'][i]= df_opp['GEOID'][j]
            break
##    if (i%1000==0)and(df_mesazone['OppZoneGeoID'][i]!='0'): print(i,df_mesazone['OppZoneGeoID'][i])      


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


13000 08077000800
14000 08077000800
15000 08077000900
17000 08077000200
19000 08077000800
21000 08077000800
25000 08077000200
28000 08077000800
29000 08077000700
30000 08077000500
31000 08077000800
34000 08077000500
37000 08077000400
41000 08077000400
47000 08077000400
48000 08077000400


,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
49844,39.089047,-108.553640,1156.0,BOOKCLIFF AVE,12,GRAND JUNCTION,CO,81501,1156.0 BOOKCLIFF AVE,08077000500,False,N/A
49845,39.087296,-108.471576,550.0,WARRIOR WAY,302,GRAND JUNCTION,CO,81504,550.0 WARRIOR WAY,00000000000,False,N/A
49846,39.088367,-108.521428,2864.0,PRESLEY AVE,NaN,GRAND JUNCTION,CO,81501,2864.0 PRESLEY AVE,00000000000,False,N/A
49847,39.087289,-108.471221,550.0,WARRIOR WAY,312,GRAND JUNCTION,CO,81504,550.0 WARRIOR WAY,00000000000,False,N/A
49848,NaN,-108.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000000000,False,N/A


In [0]:
### Match enterprise zones:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_ent)): 
        if df_ent['geometry'][j].contains(point):
            df_mesazone['EntZone'][i]= True
            break
    if (i%1000==0)and(df_mesazone['EntZone'][i]==True): print(i,df_mesazone['EntZone'][i])  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


2000 True
3000 True
4000 True
5000 True
6000 True
7000 True
8000 True
9000 True
10000 True
12000 True
13000 True
14000 True
15000 True
16000 True
17000 True
18000 True
19000 True
20000 True
21000 True
22000 True
23000 True
24000 True
25000 True
26000 True
27000 True
28000 True
29000 True
30000 True
31000 True
32000 True
33000 True
34000 True
35000 True
36000 True
37000 True
38000 True
39000 True
41000 True
42000 True
44000 True
45000 True
46000 True
47000 True
48000 True
49000 True


In [0]:
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_mesazone,'Mesa_Zone')
## df_mesazone = pd.read_csv('Mesa_Zone.csv',dtype={"StreetNumber":str,'OppZoneGeoID':str})
## df_mesazone.head()

,Latitude,Longitude,StreetNumber,StreetName,Unit,City,State,ZIP,FullAddress,OppZoneGeoID,EntZone,ZoningCode
0,38.523159,-108.889763,56500,HWY 141,NaN,GATEWAY,CO,81522,56500 HWY 141,00000000000,False,0
1,38.534833,-108.896495,55002,HWY 141,NaN,GATEWAY,CO,81522,55002 HWY 141,00000000000,False,0
2,38.535193,-108.838723,34165,S 12 8/10 RD,NaN,WHITEWATER,CO,81527,34165 S 12 8/10 RD,00000000000,False,0
3,38.541657,-108.567979,30200,UNCOMPAHGRE DIVIDE RD,NaN,WHITEWATER,CO,815XX,30200 UNCOMPAHGRE DIVIDE RD,00000000000,False,0
4,38.563368,-108.921568,52248,HWY 141,NaN,GATEWAY,CO,81522,52248 HWY 141,00000000000,False,0


In [0]:
### Match zoning:
for i in range(len(df_mesazone)):
    point=Point(df_mesazone['Longitude'][i],df_mesazone['Latitude'][i])
    for j in range(len(df_zoning_gj)): 
        if df_zoning_gj['geometry'][j].contains(point):
            df_mesazone['ZoningCode'][i]= df_zoning_gj['Name'][j]
            if i%1000==0: print(i,j,df_zoning_gj['Name'][j])
            break  

In [0]:
''' Point Check
point=Point(-108.56177,39.0676422)
for j in range(len(df_zoning_gj)):  ##1005 records, 10s per point
    if df_zoning_gj['geometry'][j].contains(point):
        print(df_zoning_gj['Name'][j])
df_mesazone.iloc[1109]
'''

In [0]:
print(df_mesazone.tail(),'\n','---------','\n')
print('Shape of the current dataframe:',df_mesazone.shape,'\n')
print('Missing value counts:')
print(df_mesazone.shape[0]-df_mesazone.count())

## Task 3 Find zoning for each Denver County address

In [0]:
for i in range(len(df_mesazone)):
    if (df_mesazone['OppZoneGeoID'][i]=='00000000000'): df_mesazone['OppZoneGeoID'][i]=='0'
    if (df_mesazone['ZoningCode'][i]=='N/A'): df_mesazone['ZoningCode'][i]=='0'

## Task 4 Combine datasets

In [0]:
df_zone=pd.concat([df_mesazone,df_denzone])
print(df_zone.tail(),'\n')
print('Shape of the current dataframe:',df_zone.shape)

           City            FullAddress   Latitude   Longitude State  \
86069  COLLBRAN   2318 DIVIDE CREEK RD  39.356364 -107.585212    CO   
86070  COLLBRAN        75239 E HWY 330  39.359693 -107.678560    CO   
86071  COLLBRAN  24550 DIVIDE CREEK RD  39.361212 -107.588424    CO   
86072  COLLBRAN        75988 E HWY 330  39.364550 -107.643852    CO   
86073  COLLBRAN        75995 E HWY 330  39.365940 -107.646237    CO   

            StreetName StreetNumber    ZIP  
86069  DIVIDE CREEK RD         2318  816XX  
86070        E HWY 330        75239  816XX  
86071  DIVIDE CREEK RD        24550  816XX  
86072        E HWY 330        75988  816XX  
86073        E HWY 330        75995  816XX   

Shape of the current dataframe: (403926, 8)


In [0]:
## Save/Checkpoint
def write_checkpoint(df, dfname):
  filename=time.strftime("%Y%m%d-%H%M")+' '+ dfname+'.csv'
  df.to_csv(filename, index=False)
  return

def read_checkpoint(filename):
    return pd.read_csv(filename,index_col=0)
  
write_checkpoint(df_mesazone,'Mesa_Zone')
write_checkpoint(df_denzone,'Denver_Zone')
write_checkpoint(df_zone,'Both_Zone')

###### df_mesazone=read_checkpoint('20190331-0937 Mesa_Zone.csv')
###### df_mesazone.head()
